In [2]:
from models import ALSRecommender, PopRecommender
from evaluation import evaluate
from processing import *
from utils import IDMAP
import pandas as pd



C:\Users\tanch\anaconda3\envs\dp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### process

In [3]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Team Assignment - book recommendation\data\Ratings.csv\Ratings.csv"
data = pd.read_csv(path)

In [8]:
data

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [2]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Team Assignment - book recommendation\data\Ratings.csv\Ratings.csv"
data = pd.read_csv(path)

# remove samples with less than minimum_interaction
processed_data = filter_by_num_interactions(data, "User-ID", "ISBN", minimum_interaction = 10)
processed_data = filter_by_num_interactions(processed_data, "ISBN", "User-ID", minimum_interaction = 10)
processed_data = filter_by_num_interactions(processed_data, "User-ID", "ISBN", minimum_interaction = 10)
processed_data = filter_by_num_interactions(processed_data, "ISBN", "User-ID", minimum_interaction = 10)
# add column containing all ones
processed_data = add_interaction_column(processed_data)
# train_test_split
train, test = train_test_split(processed_data, train_frac = 0.7, seed = 0)
# convert to csr interaction matrix i.e input to model
train_csr, users, items = df2interact_mat(df = train, user_col = 'User-ID', item_col = "ISBN", interact_col = "interaction")
# retain a mapping between userids, itemids and their indexes in the recommender model
user_map = IDMAP(users)
item_map = IDMAP(items)

In [3]:
model_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4125 - Developing Data Products\Assignments\Team Assignment - book recommendation\code\models\test0"
config_dict = {
    "factors":64, 
    "regularization":0.01, 
    "alpha":1.0, 
    "iterations" : 20,
    "calculate_training_loss":True,
    "random_state":0
}

### train and evaluate recommender model

In [4]:
# train model
model = ALSRecommender(config_dict, train_csr, user_map, item_map)
model.fit()
# save model
model.save(model_path)

100%|████████████████████████████████████████████████████████████████████| 20/20 [00:19<00:00,  1.05it/s, loss=0.00218]


In [9]:
# load model
model = ALSRecommender(config_dict, train_csr, user_map, item_map)
model.load(model_path)

In [10]:
evaluate(model, test)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1002.28it/s]


0.06068474744054424

### evaluate popularity recommender

In [11]:
pop = PopRecommender()
pop.fit(train)

In [12]:
evaluate(pop, test)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 139.95it/s]


0.0064396601308490346